In [7]:
# !pip install umap-learn

In [1]:
import sys
sys.path.append("..")
import matplotlib.pyplot as plt
from scziDesk_preprocess import *
from scziDesk_network import *
from scziDesk_utils import *
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from collections import Counter
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, KernelPCA, LatentDirichletAllocation, SparsePCA, FastICA, NMF
from sklearn.decomposition import MiniBatchDictionaryLearning
import h5py
import time
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, silhouette_score, calinski_harabasz_score
import glob2
import umap
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,
Using TensorFlow backend.


In [2]:
#"lda","nmf", 
reduction_models = ["umap"#"minibatch","pca", "kernelpca",  "sparsepca", "fastica", 
                   ]
model = reduction_models[-1]
print(model)

umap


In [3]:
for category in ["real_data", #"balanced_data",  "imbalanced_data",#
                ]:

    path= ".."
    if category in ["balanced_data", "imbalanced_data"]:
        files = glob2.glob(f'{path}/R/simulated_data/{category}/*.h5')
        files = [f[len(f"{path}/R/simulated_data/{category}/"):-3] for f in files]
    else:
        files = glob2.glob(f'{path}/real_data/*.h5')
        files = [f[len(f"{path}/real_data/"):-3] for f in files]
    print(files)

    df = pd.DataFrame(columns = ["dataset", "ARI", "NMI", "sil", "run", "time", "pred", "cal", "model"])
    for dataset in files:
        if category in ["balanced_data", "imbalanced_data"]:
            data_mat = h5py.File(f"{path}/R/simulated_data/{category}/{dataset}.h5","r")
        else:
            data_mat = h5py.File(f"{path}/real_data/{dataset}.h5","r")

        Y = np.array(data_mat['Y'])
        X = np.array(data_mat['X'])
        print(f">>>>dataset {dataset}")
        if X.shape[0] > 10000:
            continue
        n_clusters =  np.unique(Y).shape[0]

        X = np.ceil(X).astype(np.int)
        orig_X = X.copy()

        adata = sc.AnnData(X)
        adata.obs['Group'] = Y
        adata = normalize(adata,
                          copy=True,
                          highly_genes=500,
                          size_factors=True,
                          normalize_input=True,
                          logtrans_input=True)
        X = adata.X.astype(np.float32)
        Y = np.array(adata.obs["Group"])
        for model in reduction_models:
            for run in range(3):
                start = time.time()
                seed = run
                np.random.seed(seed)

                if model == "pca":
                    pca_data = PCA(2, random_state = seed).fit_transform(X)
                if model == "kernelpca":
                    pca_data = KernelPCA(2, kernel= 'cosine',random_state = seed).fit_transform(X)
                if model == "lda":
                    pca_data = LatentDirichletAllocation(2, random_state = seed).fit_transform(orig_X)
                if model == "sparsepca":
                    pca_data = SparsePCA(2, random_state = seed).fit_transform(X)
                if model == "fastica":
                    pca_data = FastICA(2, random_state = seed).fit_transform(X)
                if model == "nmf":
                    pca_data = NMF(2, random_state = seed).fit_transform(X)
                if model == "minibatch":
                    dict_learner = MiniBatchDictionaryLearning(
                        n_components=2, transform_algorithm='lasso_lars', random_state=run,
                    )
                    pca_data = dict_learner.fit_transform(X)
                if model == "umap":
                    reducer = umap.UMAP(n_components=2)
                    pca_data = reducer.fit_transform(X)



                kmeans = KMeans(n_clusters=n_clusters, init="k-means++", random_state=seed)
                pred = kmeans.fit_predict(pca_data)

                elapsed = time.time() - start
                ARI = adjusted_rand_score(Y, pred)
                NMI = np.around(normalized_mutual_info_score(Y, pred), 5)
                ss = silhouette_score(pca_data,pred)
                cal = calinski_harabasz_score(pca_data, pred)

                df.loc[df.shape[0]] = [
                    dataset, ARI, NMI, ss, run, elapsed, pred, cal, model
                ]
                df.to_pickle(f"../output/pickle_results/{category}/{category}_kernel_pca_kmeans_scalins.pkl")

['Quake_Smart-seq2_Trachea', 'Quake_Smart-seq2_Diaphragm', 'Quake_10x_Spleen', 'Young', 'mouse_ES_cell', 'Adam', 'Quake_10x_Bladder', 'Quake_Smart-seq2_Lung', 'Quake_10x_Limb_Muscle', 'worm_neuron_cell', 'mouse_bladder_cell', 'Romanov', 'Quake_Smart-seq2_Limb_Muscle', 'Muraro', '10X_PBMC']
>>>>dataset Quake_Smart-seq2_Trachea


/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


>>>>dataset Quake_Smart-seq2_Diaphragm


/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


>>>>dataset Quake_10x_Spleen


...

/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


>>>>dataset Muraro


/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


>>>>dataset 10X_PBMC


/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


In [4]:
df.groupby("model").mean()

,ARI,NMI,sil,time,cal
model,,,,,
umap,0.481887,0.585774,0.516166,8.501845,11283.911687


In [6]:
df.groupby("model").mean()

,ARI,NMI,sil,time,cal
model,,,,,
fastica,0.397478,0.539439,0.615399,0.129211,11340.241435
kernelpca,0.442428,0.583714,0.481171,0.239965,8655.126244
minibatch,0.369752,0.512282,0.692206,1.711916,11409.351856
pca,0.396959,0.538770,0.612963,0.094226,11684.671874
sparsepca,0.396504,0.536758,0.642719,8.404948,12520.505358


In [ ]:
# Real
# rbf 0.376284
# linear 0.396959
# cosine 0.442428
# poly 0.1

# Balanced
# rbf 0.215114
# linear 0.218888
# cosine 0.205289
# poly

# imBalanced
# rbf 
# linear  0.206407
# cosine 
# poly

In [ ]:
df.mean()

In [ ]:
df.groupby("dataset").mean()